Cohen manuscript zenodo data have already been downloaded by the cohen retina preprocess script.

HOWEVER the data we want (mixed_cell_pBC_exp_rep1.csv, mixed_cell_pBC_exp_rep2.csv) is not in the zenodo. While there are files with those names, they are duplicates of 

After some personal correspondance with the authors, we discovered that the real files were on NCBI

obsidian://open?vault=science&file=Excalidraw%2Fcohen_data_map_LKGLKarSVFqsg0mh0TID7KgPed2HxVRM



In [2]:
data_root="/home/mcn26/palmer_scratch/tabula_data"
output_root="/home/mcn26/palmer_scratch/tabula_data/formatted"

In [44]:
import urllib.request

urllib.request.urlretrieve("https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM5687483&format=file&file=GSM5687483%5Fmixed%5Fcell%5FpBC%5Fexp%5Frep1%2Ecsv%2Egz",
                           f"{data_root}/GSM5687483_mixed_cell_pBC_exp_rep1.csv.gz")

urllib.request.urlretrieve("https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSM5687484&format=file&file=GSM5687484%5Fmixed%5Fcell%5FpBC%5Fexp%5Frep2%2Ecsv%2Egz",
                           f"{data_root}/GSM5687484_mixed_cell_pBC_exp_rep2.csv.gz")

('/home/mcn26/palmer_scratch/tabula_data/GSM5687484_mixed_cell_pBC_exp_rep2.csv.gz',
 <http.client.HTTPMessage at 0x14e3560722c0>)

In [45]:
import pandas as pd

In [46]:
rep1=pd.read_csv(f"{data_root}/GSM5687483_mixed_cell_pBC_exp_rep1.csv.gz")

In [47]:
rep2=pd.read_csv(f"{data_root}/GSM5687484_mixed_cell_pBC_exp_rep2.csv.gz")

In [48]:
rep1["rep_id"]=1
rep2["rep_id"]=2

In [49]:
df=pd.concat([rep1,rep2],axis=0,ignore_index=True)

In [50]:
del rep1
del rep2

In [51]:
df

,cellBC,prom_id,pBC,norm_exp,direct_exp,rep_id
0,ATTCCTATCATCTGTT,chr8_56986998_56987131_-,GCATATCGGTGA,930.009780,29,1
1,ACCCAAATCCACGTAA,chr17_5522677_5522810_-,CATCGGTGAATT,1285.437156,269,1
2,CTGAATGGTTCTTGCC,chr15_100882113_100882246_-,TTGTCCGCCGTT,1133.919307,140,1
3,GTGTTCCAGAGTGTTA,chr19_24269928_24270061_+,ACGAAGTCCTGG,1310.250381,52,1
4,AGTTAGCCAGTGTATC,chr21_34398176_34398309_+,CGATTACAGAAC,1047.647048,73,1
...,...,...,...,...,...,...
782476,GAACGTTGTCCGATCG,chr4_1166304_1166437_-,CACGAACCTACT,708.174905,1,2
782477,CAGCAGCTCCAACACA,chr11_76092300_76092433_+,CACGGTACAGTT,720.896947,1,2
782478,TGCGACGCAACTTGCA,chrX_110339352_110339485_+,AGGAATCTGGTC,623.995181,1,2
782479,CGATGGCAGCGACAGT,chr11_5646168_5646301_+,TTGTGCGCCTCT,596.917584,1,2


Is that UMIs or reads..?

Ok, after a deep dive into https://github.com/barakcohenlab/scMPRA/blob/68cd4ed1b28909100507369be0fcc9e37c2d8bb2/scripts/cell_bcs_filtering.py#L37 and https://github.com/barakcohenlab/scMPRA/blob/68cd4ed1b28909100507369be0fcc9e37c2d8bb2/scripts/sc_crs_exp.py#L231 , I'm confident that `direct_exp` is `exp` is "sum of # of unique UMIs across all barcodes for a particular promoter-cell combo".

In [52]:
df=df.drop(["pBC","norm_exp"],axis=1)

In [53]:
df=df.rename({'prom_id':'CRE_id','direct_exp':'UMIs_MPRA_BC'},axis=1)
df

,cellBC,CRE_id,UMIs_MPRA_BC,rep_id
0,ATTCCTATCATCTGTT,chr8_56986998_56987131_-,29,1
1,ACCCAAATCCACGTAA,chr17_5522677_5522810_-,269,1
2,CTGAATGGTTCTTGCC,chr15_100882113_100882246_-,140,1
3,GTGTTCCAGAGTGTTA,chr19_24269928_24270061_+,52,1
4,AGTTAGCCAGTGTATC,chr21_34398176_34398309_+,73,1
...,...,...,...,...
782476,GAACGTTGTCCGATCG,chr4_1166304_1166437_-,1,2
782477,CAGCAGCTCCAACACA,chr11_76092300_76092433_+,1,2
782478,TGCGACGCAACTTGCA,chrX_110339352_110339485_+,1,2
782479,CGATGGCAGCGACAGT,chr11_5646168_5646301_+,1,2


Looks good. Let's add in cell-type information.

In [54]:
cell_type_key=pd.read_csv(f'{zenodo_root}/data/sc_cellBC_association/mixed_cell_identity.csv')

In [55]:
cell_type_key

,Unnamed: 0,leiden
0,AAACCCATCGACCACG-1-0,0
1,AAACGAACACAGTGTT-1-0,0
2,AAACGAAGTCCGGCAT-1-0,0
3,AAACGAAGTTAATCGC-1-0,0
4,AAACGAATCACTGGGC-1-0,0
...,...,...
3489,TTTGATCCATTGTGCA-1-1,0
3490,TTTGATCTCGACCCAG-1-1,0
3491,TTTGGAGAGCGACATG-1-1,0
3492,TTTGGAGCATGACGAG-1-1,0


So we have two problems.
1. Not clear what mysterious 0, 1 on the end of the cellBC represent in terms of replicate. I assume, but don't know, that replicate 0 is "rep 1" and 1 is "rep 2"
2. Not clear which leiden IDs represent which cell-types.

Let's deal with #1 first. 

In [56]:
first_mystery_index=cell_type_key['Unnamed: 0'].apply(lambda x : x.split("-")[1])
second_mystery_index=cell_type_key['Unnamed: 0'].apply(lambda x : x.split("-")[2])

In [57]:
print(first_mystery_index.unique())
print(second_mystery_index.unique())

['1']
['0' '1']


First index has no information, second index probably encodes the replicate.

In [58]:
cell_type_key['cellBC']=cell_type_key['Unnamed: 0'].apply(lambda x : x.split("-")[0])
cell_type_key['replicate']=cell_type_key['Unnamed: 0'].apply(lambda x : x.split("-")[2])
cell_type_key

,Unnamed: 0,leiden,cellBC,replicate
0,AAACCCATCGACCACG-1-0,0,AAACCCATCGACCACG,0
1,AAACGAACACAGTGTT-1-0,0,AAACGAACACAGTGTT,0
2,AAACGAAGTCCGGCAT-1-0,0,AAACGAAGTCCGGCAT,0
3,AAACGAAGTTAATCGC-1-0,0,AAACGAAGTTAATCGC,0
4,AAACGAATCACTGGGC-1-0,0,AAACGAATCACTGGGC,0
...,...,...,...,...
3489,TTTGATCCATTGTGCA-1-1,0,TTTGATCCATTGTGCA,1
3490,TTTGATCTCGACCCAG-1-1,0,TTTGATCTCGACCCAG,1
3491,TTTGGAGAGCGACATG-1-1,0,TTTGGAGAGCGACATG,1
3492,TTTGGAGCATGACGAG-1-1,0,TTTGGAGCATGACGAG,1


In [59]:
cell_type_key=cell_type_key.drop(["Unnamed: 0"],axis=1)
cell_type_key=cell_type_key.rename({'replicate':'rep_id'},axis=1)

To determine which replicate is which, I will try merging both ways, and see which has the better overlap.

In [60]:
sensible=cell_type_key.copy()
backwards=cell_type_key.copy()

In [61]:
sensible["rep_id"]=sensible["rep_id"].astype(int)+1

In [62]:
backwards["rep_id"]=backwards["rep_id"].astype(int).replace(0,2)

In [63]:
sensible_overlap=df.merge(sensible,on=["rep_id","cellBC"],how="left")
backwards_overlap=df.merge(backwards,on=["rep_id","cellBC"],how="left")

In [66]:
sum(~ sensible_overlap["leiden"].isna())

735993

In [68]:
sum(~ backwards_overlap["leiden"].isna())

1482

Seems clear that "sensible" is what they went with.

In [70]:
sensible_overlap=sensible_overlap.dropna()

Examining `zenodo_root/notebooks/Part1_section1_analyze_scTranscriptome_for_mixed_cells.ipynb` we find `cluster_annnotation = {'0':'Hek 293', '1':'K562', '2': 'Ambiguous', '3': 'Ambiguous'}`

In [75]:
cluster_annnotation = {'0':'Hek 293', '1':'K562', '2': 'Ambiguous', '3': 'Ambiguous'}
sensible_overlap["cell_type_annotation"]=sensible_overlap["leiden"].apply(lambda x : cluster_annnotation[str(int(x))])

/tmp/ipykernel_130093/2553612829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensible_overlap["cell_type_annotation"]=sensible_overlap["leiden"].apply(lambda x : cluster_annnotation[str(int(x))])


In [76]:
sensible_overlap

,cellBC,CRE_id,UMIs_MPRA_BC,rep_id,leiden,cell_type_annotation
0,ATTCCTATCATCTGTT,chr8_56986998_56987131_-,29,1,0.0,Hek 293
1,ACCCAAATCCACGTAA,chr17_5522677_5522810_-,269,1,0.0,Hek 293
3,GTGTTCCAGAGTGTTA,chr19_24269928_24270061_+,52,1,0.0,Hek 293
4,AGTTAGCCAGTGTATC,chr21_34398176_34398309_+,73,1,3.0,Ambiguous
5,TTCCAATGTTCGGACC,chr9_136215006_136215139_+,147,1,0.0,Hek 293
...,...,...,...,...,...,...
782476,GAACGTTGTCCGATCG,chr4_1166304_1166437_-,1,2,0.0,Hek 293
782477,CAGCAGCTCCAACACA,chr11_76092300_76092433_+,1,2,0.0,Hek 293
782478,TGCGACGCAACTTGCA,chrX_110339352_110339485_+,1,2,0.0,Hek 293
782479,CGATGGCAGCGACAGT,chr11_5646168_5646301_+,1,2,0.0,Hek 293


In [77]:
sensible_overlap=sensible_overlap.drop("leiden",axis=1)

,cellBC,rep_id,CRE_id,cell_type_annotation,UMIs_MPRA_BC
0,ATTCCTATCATCTGTT,1,chr8_56986998_56987131_-,Hek 293,29
1,ACCCAAATCCACGTAA,1,chr17_5522677_5522810_-,Hek 293,269
3,GTGTTCCAGAGTGTTA,1,chr19_24269928_24270061_+,Hek 293,52
4,AGTTAGCCAGTGTATC,1,chr21_34398176_34398309_+,Ambiguous,73
5,TTCCAATGTTCGGACC,1,chr9_136215006_136215139_+,Hek 293,147
...,...,...,...,...,...
782476,GAACGTTGTCCGATCG,2,chr4_1166304_1166437_-,Hek 293,1
782477,CAGCAGCTCCAACACA,2,chr11_76092300_76092433_+,Hek 293,1
782478,TGCGACGCAACTTGCA,2,chrX_110339352_110339485_+,Hek 293,1
782479,CGATGGCAGCGACAGT,2,chr11_5646168_5646301_+,Hek 293,1


In [78]:
sensible_overlap.to_csv(f"{output_root}/COHEN_MIXED_CELL.tsv",sep="\t")